## Reproducing and analyzing the results of the article "[MODEL EDITING AS A ROBUST AND DENOISED VARI- ANT OF DPO: A CASE STUDY ON TOXICITY](https://arxiv.org/pdf/2405.13967)"

В данном ноутбуке мы хотим воспроизвести полученные в статье результаты, проанализировать применные авторами подходы. Оценить преимущества и недостатки:

В статье основной эксперимент првоодили на GPT-2-Medium, мы тоже будем на ней из-за ее небольшого размера 355M параметров. Выбор параметров модели:

* __pref_data_dps__ - pairwise toxic data сгенерированные в [статье](https://arxiv.org/abs/2401.01967)<br>
"The non-toxic sequences are extracted from Wikitext-2, and their toxic counterparts are generated using PPLM" - Данные взяты из статьи, в репозитории которой не содержится кода по получению пар данных с применением PPLM, лишь одноабзацное описание с упоминанием гиперпараметров обучения GPT-2.
<br> Это первая задача проекта - нужно разобраться с генерацией PPLM так как это позволит сгенерировать свои данные разных доменов (не только токсичности, но и например нарушения против религий, расы и тд).<br><br>
* __centering__ - параметр центрирования, который удаляет фоновый (частотный) шум, улучшая внимание модели на более редкие и значимые компоненты, которые связаны с токсичным содержанием. Детали будут ниже.<br><br>
* __edit_keys__ - фильтр к MLP-Key, причем в Readme авторы написали, что рекомендуют не изменять, так как это не снижает токсичность. В самой статье об этом не упоминается, но если задуматься MLP-Key используется для создания "ключей" (keys), которые необходимы для механизма внимания.<br>
Ключи определяют, какая часть информации (то есть какие значения) будет выбрана и как сильно она будет влиять на итоговое представление. Применение фильтра к MLP-Key может нарушить корректную работу механизма внимания. Однако сами ключи не несут содержательной информации, которая непосредственно появляется в выходном тексте.
Ключи используются для определения релевантности информации, и их изменение может нарушить точность сопоставления, что приведёт к потере эффективности и точности работы модели.
<br>Поскольку MLP-Key играет вспомогательную роль и не влияет на непосредственное содержание, фильтрация здесь не имеет смысла — она не повлияет на токсичность напрямую.<br><br>
* __edit_values__ - фильтр к MLP-Value<br><br>
* __lowest_layer_to_edit__ и __top_k_ranks__ - являются ключевыми гиперпараметрами алгоритма, которые подбирались авторами.<br><br>
__k__ — это число правых сингулярных векторов, которые используются для построения проекционной матрицы, нацеленное на выделение токсичного подпространства. Эти векторы помогают определить основные токсичные направления в пространстве эмбеддингов. Чем больше k, тем больше направлений учитывается в токсичном подпространстве. Кода с получением этого значения нет. У GPT-2 это значение довольно низкое - 2, в том время как у других рассмаотренных моделей оно взято 10.<br><br>
__L__ — это номер слоя, с которого начинается процесс редактирования.<br>
Важно выбрать правильный слой для начала редактирования, так как в верхних слоях модели обрабатываются более сложные и высокоуровневые концепции, такие как токсичность. Редактирование с этого уровня позволяет эффективно удалять токсичную информацию, сохраняя основную способность модели обрабатывать текст.

Пример начала анализа (не окончен):

In [1]:
# При изменениях в коде проекте эти обновления будут автоматически подгружаться в ноутбук
%load_ext autoreload
%autoreload 2

Настройка путей для импорта модулей:

In [ ]:
import os
import sys
import inspect
import argparse
import numpy as np
from detox import DeToxEdit
import torch

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

from utils.startup import main
from evaluate_model import evaluate_model

if '__file__' in globals():
    parser = argparse.ArgumentParser(description='DeTox')
    parser.add_argument('--config_file', default='gpt2-medium.ini', type=str, help='Config Filename. E.g. gpt2-medium.ini')
    args = parser.parse_args()
    config_filename = args.config_file
else:
    config_filename = 'gpt2-medium.ini'

config = main(config_filename=config_filename)


No CUDA_VISIBLE_DEVICES found. Setting to 0.
PROJECT_ROOT: 'detox-edit'
PYTHONPATH: 'detox-edit':
HF_HOME: 'detox-edit'


Загрузка модели GPT-2 и токенизатора

In [ ]:
from utils.model_utils import load_large_model
from detox import DeToxEdit

model_id = 'gpt2'
model, tokenizer = load_large_model(model_id)

INFO:root:Model gpt2 loaded.


Слои для изменений, указанные в статье для GPT-2, как уже упомяналось, кода по определению этих границ нет, лишь небольшое текстовое описание в самой статье

In [7]:
lower_layer = 10
upper_layer = 24

layer_range = np.arange(lower_layer, upper_layer) if lower_layer != -1 and upper_layer != -1 else None

In [8]:
editor = DeToxEdit(
    model=model,
    tokenizer=tokenizer,
    pref_data_dps=500,
    centering=True,
    top_k_ranks=2,
    edit_layer_range=layer_range,
    random_dps=True
)

Авторы статьи проводят центрирование, но при этом не указывает почему оно имеет значение, при этом допускают его в качестве параметра. Хочется понять как центрирование на самом деле влияет

In [11]:
ats, preference_matrix  = editor.get_ats()

INFO:root:Loaded 500 preferred and 500 non-preferred samples.
INFO:root:Batch 1/10 of size 50
INFO:root:Batch 2/10 of size 50
INFO:root:Batch 3/10 of size 50
INFO:root:Batch 4/10 of size 50
INFO:root:Batch 5/10 of size 50
INFO:root:Batch 6/10 of size 50
INFO:root:Batch 7/10 of size 50
INFO:root:Batch 8/10 of size 50
INFO:root:Batch 9/10 of size 50
INFO:root:Batch 10/10 of size 50
INFO:root:Hidden sent: torch.Size([24, 500, 1024])
INFO:root:Batch 1/10 of size 50
INFO:root:Batch 2/10 of size 50
INFO:root:Batch 3/10 of size 50
INFO:root:Batch 4/10 of size 50
INFO:root:Batch 5/10 of size 50
INFO:root:Batch 6/10 of size 50
INFO:root:Batch 7/10 of size 50
INFO:root:Batch 8/10 of size 50
INFO:root:Batch 9/10 of size 50
INFO:root:Batch 10/10 of size 50
INFO:root:Hidden sent: torch.Size([24, 500, 1024])
INFO:root:Preference matrix calculated.
INFO:root:Centering: Removing first singular vector from preference matrix.


In [23]:
not_center_ats, not_center_preference_matrix = editor.get_ats()

INFO:root:Loaded 500 preferred and 500 non-preferred samples.
INFO:root:Batch 1/10 of size 50
INFO:root:Batch 2/10 of size 50
INFO:root:Batch 3/10 of size 50
INFO:root:Batch 4/10 of size 50
INFO:root:Batch 5/10 of size 50
INFO:root:Batch 6/10 of size 50
INFO:root:Batch 7/10 of size 50
INFO:root:Batch 8/10 of size 50
INFO:root:Batch 9/10 of size 50
INFO:root:Batch 10/10 of size 50
INFO:root:Hidden sent: torch.Size([24, 500, 1024])
INFO:root:Batch 1/10 of size 50
INFO:root:Batch 2/10 of size 50
INFO:root:Batch 3/10 of size 50
INFO:root:Batch 4/10 of size 50
INFO:root:Batch 5/10 of size 50
INFO:root:Batch 6/10 of size 50
INFO:root:Batch 7/10 of size 50
INFO:root:Batch 8/10 of size 50
INFO:root:Batch 9/10 of size 50
INFO:root:Batch 10/10 of size 50
INFO:root:Hidden sent: torch.Size([24, 500, 1024])
INFO:root:Preference matrix calculated.
INFO:root:No centering: No removing first singular vector from preference matrix.


In [25]:
svd = editor.svd_on_ats(ats)
not_center_svd = editor.svd_on_ats(not_center_ats)

INFO:root:SVD of ATS calculated.
INFO:root:SVD of ATS calculated.


In [37]:
for layer_num in range(preference_matrix.shape[0]):
    centered_flat = preference_matrix[layer_num].flatten()
    noncentered_flat = not_center_preference_matrix[layer_num].flatten()
    correlation = torch.corrcoef(torch.stack([centered_flat, noncentered_flat]))[0, 1]
    print(f"Layer {layer_num} correlation between centered and non-centered: {correlation.item()}")


Layer 0 correlation between centered and non-centered: 0.05186564475297928
Layer 1 correlation between centered and non-centered: 0.059206850826740265
Layer 2 correlation between centered and non-centered: 0.059118837118148804
Layer 3 correlation between centered and non-centered: 0.057201966643333435
Layer 4 correlation between centered and non-centered: 0.052401527762413025
Layer 5 correlation between centered and non-centered: 0.04660128057003021
Layer 6 correlation between centered and non-centered: 0.04584299772977829
Layer 7 correlation between centered and non-centered: 0.044945117086172104
Layer 8 correlation between centered and non-centered: 0.04558739438652992
Layer 9 correlation between centered and non-centered: 0.045619018375873566
Layer 10 correlation between centered and non-centered: 0.04570295289158821
Layer 11 correlation between centered and non-centered: 0.047272924333810806
Layer 12 correlation between centered and non-centered: 0.048108987510204315
Layer 13 corre

In [ ]:
toxic_subspace = editor.find_p_toxic(svd)

In [ ]:
edited_model = editor.edit_model(toxic_subspace, edit_keys=True, edit_values=True, layer_range=layer_range)

In [ ]:
ppl, tox = evaluate_model(edited_model, tokenizer,
               return_perplexity=True, return_toxicity=True, display_gen=True)
print(f'{model_id} - Perplexity: {ppl}, Toxicity: {tox}')